# Imports 

In [1]:
"""

pip install scipy
pip install h5py
pip install matplotlib
pip install opencv-python
pip install seaborn
pip install matplotlib
pip install ipywidgets
pip install matplotlib install ipywidgets
pip install opencv-pythonjupyter nbextension enable --py widgetsnbextension
pip install ipython
pip install WordCloud
pip install geopy
pip install scipy
pip install plotly
pip install nbformat --upgrade
pip install folium
pip install geopandas
pip install scikit-learn   
pip install cairosvg
pip install lxm



pip install selenium
pip install pandas
pip install python-dotenv
pip install openai
pip install beautifulsoup4
pip install numpy
pip install tqdm
pip install seaborn
pip install matplotlib
pip install scipy
pip install geopy
pip install plotly



"""

'\n\npip install scipy\npip install h5py\npip install matplotlib\npip install opencv-python\npip install seaborn\npip install matplotlib\npip install ipywidgets\npip install matplotlib install ipywidgets\npip install opencv-pythonjupyter nbextension enable --py widgetsnbextension\npip install ipython\npip install WordCloud\npip install geopy\npip install scipy\npip install plotly\npip install nbformat --upgrade\npip install folium\npip install geopandas\npip install scikit-learn   \npip install cairosvg\npip install lxm\n\n\n\npip install selenium\npip install pandas\npip install python-dotenv\npip install openai\npip install beautifulsoup4\npip install numpy\npip install tqdm\npip install seaborn\npip install matplotlib\npip install scipy\npip install geopy\npip install plotly\n\n\n\n'

In [2]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from dotenv import load_dotenv
import os
import openai

from bs4 import BeautifulSoup
import re
import time
import pickle
import numpy as np 
import pandas as pd

from scrapifurs import utils
from scrapifurs.GPTinstructions import GPTinstructions

from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.cluster.hierarchy import linkage, leaves_list

from geopy.geocoders import Nominatim
from geopy.distance import geodesic


import plotly.express as px

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime





In [3]:
def get_lat_lon(locations):
    geolocator = Nominatim(user_agent="example.lastname@gmail.com")
    lat_lon_data = {}
    for location in tqdm(locations):
        location_data = geolocator.geocode(location)
        if location_data is not None:
            lat_lon_data[location] = (location_data.latitude, location_data.longitude)
        else:
            lat_lon_data[location] = (None, None)
    return lat_lon_data

def calculate_distances(lat_lon_data, target_loc_str):
    target_location = get_lat_lon([target_loc_str])[target_loc_str]
    if target_location[0] is None:
        raise ValueError("No latitude and longitude found for the provided location.")
    distances = {}
    for location, loc_lat_lon in lat_lon_data.items():
        if loc_lat_lon[0] is not None:
            distance = geodesic(target_location, loc_lat_lon).miles
            distances[location] = distance
        else:
            distances[location] = None
    return distances





def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    union_length = len(set1.union(set2))
    if union_length == 0:
        return np.nan
    return len(set1.intersection(set2)) / union_length


class GPT_StringDataCleaner:
    def __init__(self, original_df, key_name, instructions, overwrite_cleaned_key = True, model="gpt-4"):
        self.original_df = original_df
        self.key_name = key_name
        self.instructions = instructions
        self.model = model
        self.clean_key = key_name + '_CLEANED_BY_GPT'
        self.overwrite_cleaned_key = overwrite_cleaned_key

    def send_to_gpt(self):
        self.unique_names = sorted(self.original_df[self.key_name].unique())

        messages = [{"role": "system", "content": self.instructions}]
        messages.append({"role": "user", "content": str(self.unique_names)})
        chat = openai.ChatCompletion.create(model=self.model, messages=messages)
        self.reply = chat.choices[0].message.content
    def test_lengths(self):
        print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
    def process_data(self):
        cleaned_data =  eval(self.reply)
        
        if len(self.unique_names) != len(cleaned_data):
            print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
            raise ValueError("Input and output lists must have the same length")

        temp_df = pd.DataFrame({self.key_name: self.unique_names, self.clean_key: cleaned_data})
        # delete cleaned key data if we want to overwrite
        if self.clean_key in self.original_df.columns and self.overwrite_cleaned_key:
            self.original_df = self.original_df.drop(self.clean_key, axis=1)
        if self.clean_key not in self.original_df.columns:
            merged_df = pd.merge(self.original_df, temp_df[[self.clean_key, self.key_name]],
                                 on=self.key_name, how='left')
            self.original_df[self.clean_key] = merged_df[self.clean_key]
            merged_df = None

        return self.original_df

In [4]:

def update_master_df(job_data_all, job_data):
    # Reverse the order of job_data to check oldest entries first
    job_data_reversed = job_data.iloc[::-1]
    
    # List to hold new entries
    new_entries = []
    
    for i, row in job_data_reversed.iterrows():
        # Check if row exists in job_data_all
        is_exist = job_data_all[(job_data_all['job_title'] == row['job_title']) & 
                                (job_data_all['company_name'] == row['company_name']) &
                                (job_data_all['location'] == row['location'])].shape[0]
        
        # If row does not exist in job_data_all, add it to new_entries
        if is_exist == 0:
            print('NEWWWWWW')
            print(row)
            new_entries.append(row)
    
    # Convert new_entries to DataFrame and concatenate it with job_data_all
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        job_data_all = pd.concat([new_entries_df, job_data_all], ignore_index=True)
    
    return job_data_all


def get_job_blocks(gpt_text, job_data):
    job_blocks = []
    combined_titles = "|".join([re.escape(title) for title in job_data['job_title']])
    combined_companies = "|".join([re.escape(company) for company in job_data['company_name']])
    pattern = f"({combined_titles}).*?({combined_companies}).*?(?=({combined_titles}).*?({combined_companies})|$)"
    
    for match in re.finditer(pattern, gpt_text, re.DOTALL):
        job_blocks.append(match.group(0))

    return job_blocks

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def go_to_next_page(driver):
    try:
        # Step 1: Find the pagination element
        pagination = driver.find_element(By.CSS_SELECTOR, 'ul.artdeco-pagination__pages')
    except NoSuchElementException:
        print("Pagination not found")
        return

    try:
        # Step 2: Find the current active page
        current_page_elem = pagination.find_element(By.CSS_SELECTOR, 'li.active')
        current_page = int(current_page_elem.text)
    except NoSuchElementException:
        print("Current page not found")
        return

    # Step 3: Find the next page
    next_page = current_page + 1
    next_page_selector = f'li[data-test-pagination-page-btn="{next_page}"]'

    try:
        next_page_elem = pagination.find_element(By.CSS_SELECTOR, next_page_selector)
    except NoSuchElementException:
        print("Next page not found")
        return

    # Step 4: Click the next page
    next_page_elem.click()


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

def update_dataframe(df_main, df_new, keys=None):
    if keys is None:
        keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']

    # Check if types are the same for each key
    mismatched_keys = []
    for key in keys:
        type_main = df_main[key].dtype
        type_new = df_new[key].dtype
        if type_main != type_new:
            mismatched_keys.append((key, type_main, type_new))

    if mismatched_keys:
        print("Type mismatch for these keys:")
        for key, type_main, type_new in mismatched_keys:
            print(f"Key: {key}, Type in df_main: {type_main}, Type in df_new: {type_new}")
        raise TypeError("Type mismatch detected.")
        
    for _, new_row in df_new.iterrows():
        mask = df_main[keys].eq(new_row[keys]).all(axis=1)
        if not any(mask):
            df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
            os.system('say "youve got jobs"')  # macOS
            print('\nNNNNNEEWWWWWWW')
            print(new_row['job_link'])
            print(new_row)

    return df_main


# def update_dataframe(df_main, df_new, keys=None):
#     if keys is None:
#         # keys = df_main.columns.tolist()
#         keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']
        
        
#     for i, new_row in df_new.iterrows():
#         if not any(df_main[keys].eq(new_row[keys]).all(axis=1)):
            
#             df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
#             print('\nNNNNNEEWWWWWWW')
#             print('\n')
#             print(new_row['job_link'])
#             print(new_row)
#             asdfasdfasdf
#     return df_main




### INIT API key,  intrucitons for GPT which can be saved as text files in the correct data/instrucitons dir of the package

In [6]:

# # setup API key for chatGPT 
# load_dotenv()  # take environment variables from .env.
# os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")
# openai.api_key = os.environ["OPENAI_API_KEY"]




# Info_dict to set up all used variables

In [7]:
# setup basic variable as dict 
info_dict = {'init_url':'https://www.linkedin.com/',
             'save_password_dir':'/Users/phil/Dropbox/GITHUB/DATA/scrapifurs/saved_cookies/',
             'start_url':'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'}
info_dict['full_cookies_save_path'] = info_dict['save_password_dir']+os.sep+"linkedin_cookies.pkl"


# Scraping step

### create rules to trim text based on key workds and search characteristics
this willl help reduce the amount of data we pass to GPT API and reduce costs 

In [8]:
# text_finder = utils.StringSectionExtractor()
# text_finder.add_start_rule('\d+ results', True)
# text_finder.add_start_rule('All filters', False)
# text_finder.add_start_rule('Jump to active job details', False)


# text_finder.add_end_rule('Page \d+ of \d+', True)
# text_finder.add_end_rule("Are you finding", False)

In [9]:
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# def click_show_more_button(driver):
#     wait = WebDriverWait(driver, 4)
    
#     # Locate the button by its aria-label
#     see_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Click to see more description"]')))
    
#     # Check if the button's content is closed
#     is_closed = see_more_button.get_attribute('aria-expanded') == 'false'
    
#     if is_closed:
#         # Scroll the button into view
#         driver.execute_script("arguments[0].scrollIntoView(true);", see_more_button)
#         time.sleep(2)  # Slight delay

#         # Try clicking the button, catch any exception and try JavaScript click as a fallback
#         try:
#             see_more_button.click()
#         except Exception as e:
#             print(f"Error in 'click_show_more_button': {e}, attempting JS click")
#             driver.execute_script("arguments[0].click();", see_more_button)
# click_show_more_button(driver)


In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time




In [12]:
# toggle_show_button(driver, expand=True)

In [21]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
from datetime import datetime
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time



def open_browser(info_dict):
    #init chrome 
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(info_dict['init_url'])
    time.sleep(1)
    driver.get(info_dict['init_url'])
    time.sleep(2)
    
    
    # Load cookies if they exist
    try:
        cookies = pickle.load(open(info_dict['full_cookies_save_path'], "rb"))
        for cookie in cookies:
            driver.add_cookie(cookie)
        driver.refresh()
        assert(not not cookies)# if empty try a different method
    except:
        print("No cookies found. Manual login required.")
        # If not logged in
        input('Please login and press Enter to continue...')
        pickle.dump(driver.get_cookies(), open(info_dict['full_cookies_save_path'], "wb")) # save cookies after login
    time.sleep(4)
    return driver
    

def update_csv(data_df, file_name):
    """
    Update the master_jobs_applied_to.csv with the provided data DataFrame.
    
    Parameters:
    - data_df: DataFrame containing the scraped data.
    """
    
    # Enforce consistent datatypes
    data_df = data_df.astype({
        'linked_in_link': 'string',
        'job_id': np.int64,
        'job_title': 'string',
        'company_name': 'string',
        'apply_link': 'string',
        'job_location': 'string',
        'time_since_posted': 'string',
        'num_applicants': 'string',
        'salary_range': 'string',
        'job_type': 'string',
        'job_level': 'string',
        'company_info': 'string',
        'description': 'string',
        'date_number': np.int64,
        'formatted_date': 'string'
    })

    # Check if master_jobs_applied_to.csv exists
    try:
        master_df = pd.read_csv(file_name)
    except FileNotFoundError:
        master_df = pd.DataFrame(columns=data_df.columns)
    # print(master_df.keys())
    # print(data_df.keys())

    # Check for new entries based on 'job_id'
    new_entries = data_df[~data_df['job_id'].isin(master_df['job_id'])]

    # If there are new entries, append them
    if not new_entries.empty:
        master_df = pd.concat([master_df, new_entries], ignore_index=True)
        master_df.to_csv(file_name, index=False)
    else:
        print("Repeat entry, ignoring.")

def find_show_more_button(driver):
    try:
        return WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Click to see more description"]'))
        )
    except:
        return None

def find_show_less_button(driver):
    try:
        return WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Click to see less description"]'))
        )
    except:
        return None
def click_show_more_button(driver):
    toggle_show_button(driver, expand=True)
    
def toggle_show_button(driver, expand):
    """
    Toggle the 'See more' or 'See less' button based on the desired state.

    :param driver: WebDriver instance
    :param expand: True to expand 'See more', False to collapse 'See less'
    """
    if expand:
        # We want to expand the description
        button = find_show_more_button(driver)
        if button:
            perform_click(driver, button, "Show more")
        assert find_show_less_button(driver), "Failed to expand description"
    else:
        # We want to collapse the description
        button = find_show_less_button(driver)
        if button:
            perform_click(driver, button, "Show less")
        assert find_show_more_button(driver), "Failed to collapse description"


def perform_click(driver, button, button_name):
    driver.execute_script("arguments[0].scrollIntoView(true);", button)
    time.sleep(2)  # Slight delay
    try:
        button.click()
    except Exception as e:
        # print(f"Error clicking '{button_name}' button: {e}, attempting JS click")
        driver.execute_script("arguments[0].click();", button)



def get_apply_link(driver):
    try:
        # Open the link in a new tab using JavaScript.
        apply_button = driver.find_element(By.CSS_SELECTOR, ".jobs-apply-button.artdeco-button")
        driver.execute_script("window.open(arguments[0].click(), '_blank');", apply_button)
        # Switch to the new tab.
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)
        # Copy the URL.
        apply_link = driver.current_url
        # Close the new tab.
        driver.close()
        # Switch back to the original tab.
        driver.switch_to.window(driver.window_handles[0])
        return apply_link
    except Exception as e:
        print(f"Error: {e}")
        return None


def get_location(element):
    location_div = element.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description div")
    location_parts = location_div.text.split('·')
    potential_location = location_parts[1].strip().split()
    if "," in potential_location[1]:
        return ' '.join(potential_location[:2])
    else:
        return ' '.join(potential_location[:1])

# Define a function to safely extract elements
def safe_extract(element, css_selector, attribute=None):
    try:
        if attribute:
            return element.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
        return element.find_element(By.CSS_SELECTOR, css_selector).text
    except NoSuchElementException:
        return None

def get_all_skills(driver):
    click_success = click_Show_all_skills_button(driver)
    if not click_success:
        return 'NA', 'NA'
    matched_skills = get_matched_skills(driver)
    unmatched_skills = get_unmatched_skills(driver)
    # click_done_button(driver)
    driver.refresh()
    
    time.sleep(4)
    return matched_skills, unmatched_skills

def click_Show_all_skills_button(driver):

    try:
        # Wait for the button to be present
        button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//button/span[text()="Show all skills"]'))
        )

        # Scroll the button into view and click
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)  # sleep can be problematic, use it with caution
        try:
            button.click()
        except:
            driver.execute_script("arguments[0].click();", button)
        time.sleep(1)  # consider using more dynamic waits instead of sleep
    except TimeoutException:
        return False  # Return None if button is not present
    return True


def get_matched_skills(driver):
    try:
        skills_elements = driver.find_elements(By.CSS_SELECTOR, ".job-details-skill-match-status-list__matched-skill div:nth-child(2)")
        return [skill.text for skill in skills_elements]
    except Exception as e:
        print(f"Error in get_matched_skills: {e}")
        return None

def get_unmatched_skills(driver):
    try:
        unmatched_skills_elements = driver.find_elements(By.CSS_SELECTOR, ".job-details-skill-match-status-list__unmatched-skill > div > div[aria-label]")
        return [skill.text for skill in unmatched_skills_elements]
    except Exception as e:
        print(f"Error in get_unmatched_skills: {e}")
        return None
        
def split_and_clean(text):
    return [line.strip() for line in text.split('\n') if line.strip()]


def check_job_id(file_name, job_id):
    try:
        df = pd.read_csv(file_name)
        return job_id in df['job_id'].values
    except FileNotFoundError:
        return False  # File doesn't exist, so job_id isn't there



def save_jobs_data(driver, url2scrape, file_name):
    # Check if url2scrape is a list
    if isinstance(url2scrape, list):
        for url in url2scrape:
            save_jobs_data(driver, url, file_name)
        return

    # Extracting data from the provided HTML
    linked_in_link = url2scrape
    job_id_match = re.search(r'/view/(\d+)/', linked_in_link)
    job_id = int(job_id_match.group(1)) if job_id_match else -1
    # check if it in there first to avoid repeating all the process
    is_repeat = check_job_id(file_name, job_id)
    print(f'JOB ID: {job_id}')
    if is_repeat:
        print('Repeat entry, skipping...')
    else:
        
    
        driver.get(url2scrape)
        
        # driver.refresh()
        time.sleep(6)
        click_show_more_button(driver)
        time.sleep(4)
        
        
    


        
        data = {
            "linked_in_link": linked_in_link,
            "job_id": job_id,
            "job_title": safe_extract(driver, '.job-details-jobs-unified-top-card__job-title'),
            "company_name": safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description a'),
            "apply_link": '',
            "job_location": get_location(driver),
            "time_since_posted": safe_extract(driver, '.tvm__text--neutral span'),
            "num_applicants": safe_extract(driver, '.tvm__text--neutral:last-child'),
            "salary_range": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight span span:first-child'),
            "job_type": safe_extract(driver, '.ui-label.ui-label--accent-3:nth-child(1) span:first-child'),
            "job_level": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight-view-model-secondary:nth-child(3)'),
            "company_info": safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight:nth-child(2) span'),
            "description": safe_extract(driver, ".jobs-description-content__text")
        }
    
        
        
        matched_skills, unmatched_skills = get_all_skills(driver)
        
        data['matched_skills'] = matched_skills
        data['unmatched_skills'] = unmatched_skills
        
        date_number = int(datetime.now().strftime('%Y%m%d'))
        
        # Get the formatted date in the format YYYY-MM-DD
        formatted_date = datetime.now().strftime('%Y-%m-%d')
        
        data['date_number'] = date_number
        data['formatted_date'] = formatted_date
        data['apply_link'] = get_apply_link(driver)
        
        
        data = {
            key: [value] for key, value in data.items()
        }
        
        df = pd.DataFrame(data)
        update_csv(df, file_name)
        print('________________________')



### INIT: chrome browser, login, save cookies (future login)

In [22]:
x = """

https://www.linkedin.com/jobs/view/3752873243/?eBP=CwEAAAGLr_TKre_U6RPEzgHUmAw4yB9J-cURb9pSR0JOgCbRni_Fn4pGiGDuVgrU2f5RSbu004YMPQy6NUh2Ou1k_2Thp6DIXxdFUJ9nuYLjsAdsH-NM1vy9IeA1j5bjj4w_4vKEgvDzPA5wxoUhyuEP1lRu7MwaNIYju52fCTNdlS199fVpqm9k1ljV92cZZV1sHgm2AA-7TxdIFKu25QRTSNOobVSRp9UGtg9_GQk4KP7AED34i_0PjOhK5HlXTAKZ1duYznGUNsshad7yIK5rOJn1JwyNju5ikSHGHrK-1ZzO_-e0WUp46fq-JX5XjBfqi0QEqvbzlMhRXW99ll1OhjWB4SABu8Ks5-_ZlSqDchUC_fO0yM-V3RJCk4XaaA&refId=22743417-4611-3b9c-9c54-43b1c50c0c43&trackingId=KHVa5cqgQI9p0OsWAxwFEw%3D%3D



https://www.linkedin.com/jobs/view/3761064412/?eBP=JOB_SEARCH_ORGANIC&refId=VeEUpzF9Bktiu6wP36LgZw%3D%3D&trackingId=KqxlwEp%2FatoW1wpMbFH%2Bog%3D%3D&trk=flagship3_search_srp_jobs


https://www.linkedin.com/jobs/view/3761150179/post-apply/default/?eBP=CwEAAAGLsDe2vvUNV_as_2O3a7HMvyizabIVV3EsASZ1HXvweXF9qksa3Z8akpiSgjENoinTVUNvphs5XH3DZxLW95ESsrmsa3b8qB7oaUfqs8O-VhrYp7ESZCherzStSc2ca2EXVZTZUMjrhqNWCqFBPEs9MqZFKhawdX3M-eg1KifXmkjzMBnZFpPpJzBvz1vH_fnRF94O_nqtN7BENFL2mlM4w5nJ4stOyGHQnFLzmL0EV-te2Z4T86sX9oV-sbxtb3m9_bhkewDmyXkV8Px3NcNTnboX2CJ7aJBfNeQoa-AY3EhRKPhWeynt85YnBIIV2VwrlxUn9sCB_mud7J7S8JELILhEnVPRMLV8IjOJcsWXubtALDWhsa1klICgYkws0hpi3J8qwLr7&postApplyJobId=3761150179&refId=l5l104x8kIUy%2FWadXpUT1g%3D%3D&trackingId=04jYxVCsixCUAes8%2BQ0zLg%3D%3D


https://www.linkedin.com/jobs/view/3759646613/?eBP=CwEAAAGLsCI-GLuxSNtP2f66VezT3qjpnSGnn1C29ixj7CM-mZAxy2zb2-mR63up41L_GnVF1zEH7rY5Nd-YsyzEhlRC3Z5Ew__uR3WBbUwPdFJoRsCKyBkDHvZD1Z_ShgwBofWJEmaOPZb3v7ofRmxktC9pvLw8IWAcIywn7Ik0_1tqGI9nheM16b1YaGMIbj6ftmCN8zUDFKXqxYO2VYXBekq8pPzStx0h3yiDqGaY5nF0O_j3VWyIc4f_SFD0x3fyXi8f4IEw2WZCbdjrqJPENBspMm2-_SlWYSTJ7lKJ3djtf9ck3xSCZK51h5yjuyz9DFzU9EVhCmWMiXYnBHdTqrmfzCekPNLJ52ZLDEl0oTRltqIQEQ3Q8EDHmJ67xCtVv_bknF8&refId=bieE3t6uhTKNPQkZIingdg%3D%3D&trackingId=9qUC4T2Ry7IWIj5b%2FWWuxA%3D%3D&trk=flagship3_search_srp_jobs


https://www.linkedin.com/jobs/view/3760763408/?eBP=CwEAAAGLsCK6FcyG_PP8U4zFn84mhsPsSsg4g6bC-LxCKOSus6UNqj4fQAGv0r3cOML_0_5pFBVtr8RaUdrvy0MNIe24m13wgkSeCn-ilzqHaHf0rpRm-xFz30I2n8Rj2cKqD_XxL1zUd2gYTX16tL5P0r78H2VuulFrwQzDXQRY5slBrhAV1YpnLU9SrJRv8bUp6tXy69luH5h17pD5StQkga9RCeIO_ZAxtGBSUnUOp1r5EqfMgdgmQd6t8X6v1KCVsAWydNswC_EjAiXXOb_Wf2HoXo_8gwQnM9mdYnXSKteOHIlRYh9GUDviroQwNLvGjv7SP0LIolfrCIWnpiz4070u2XOcANeNpYpOqrKdwnV-I50rOJg9CUe097v_03jd9diCCiiwTw&refId=P6k%2FSgBPIKPdbFVIufjDSg%3D%3D&trackingId=ndg6YVuXzYNRIg1O6PW8DQ%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3760971942/?eBP=JOB_SEARCH_ORGANIC&refId=P6k%2FSgBPIKPdbFVIufjDSg%3D%3D&trackingId=eP1XyvqhENQiS0OYcO4ZYQ%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3758749120/?eBP=CwEAAAGLsDe2vjPZQzak9u63FoAphlgHtwAsyP_zCFuKtOxUuqAQnhm9tZwgDjAh54dZpjXWDg8pqOPVDd2q8XL21Vf8H4oZMLgkIfRUgRvgnRA7ZtiTmeTQK-4auUTU5l1i9ePSSzn_d0GK4Hi4Pon_XqscCX8CPfH3rYyua1X_iKfUVorjBRBIWXPO7mecdCDrEG0w2UPsOCz6Fnefjv0DNl14u22n20ntYSxnk5ro61JZvNL2shU9U2QrTi8Totb0cbWAKQPMsvLmlArAtifwCLbpSguV5xHITw2bpv5MniJah7zC_pzHa3xnITiXxD4x8T9tAGOTjXLhWD3QeIM0FWxPyIaD9DB5I_GERYrYxDE2iCZygW7mecatn1Vu8pYReyqrvLtyEA&refId=l5l104x8kIUy%2FWadXpUT1g%3D%3D&trackingId=BXDldRJxVG2sG3MIeqHMNQ%3D%3D&trk=flagship3_search_srp_jobs



https://www.linkedin.com/jobs/view/3758749120/?eBP=CwEAAAGLsDe2vjPZQzak9u63FoAphlgHtwAsyP_zCFuKtOxUuqAQnhm9tZwgDjAh54dZpjXWDg8pqOPVDd2q8XL21Vf8H4oZMLgkIfRUgRvgnRA7ZtiTmeTQK-4auUTU5l1i9ePSSzn_d0GK4Hi4Pon_XqscCX8CPfH3rYyua1X_iKfUVorjBRBIWXPO7mecdCDrEG0w2UPsOCz6Fnefjv0DNl14u22n20ntYSxnk5ro61JZvNL2shU9U2QrTi8Totb0cbWAKQPMsvLmlArAtifwCLbpSguV5xHITw2bpv5MniJah7zC_pzHa3xnITiXxD4x8T9tAGOTjXLhWD3QeIM0FWxPyIaD9DB5I_GERYrYxDE2iCZygW7mecatn1Vu8pYReyqrvLtyEA&refId=l5l104x8kIUy%2FWadXpUT1g%3D%3D&trackingId=BXDldRJxVG2sG3MIeqHMNQ%3D%3D&trk=flagship3_search_srp_jobs


https://www.linkedin.com/jobs/view/3759308146/?eBP=JOB_SEARCH_ORGANIC&refId=bieE3t6uhTKNPQkZIingdg%3D%3D&trackingId=GBODNTi4WNxE01bKqBfmwQ%3D%3D



https://www.linkedin.com/jobs/view/3759643502/?eBP=CwEAAAGLsCI5KHqyO4q_B8z7PCXnLH_omEo3i1vsO5JbB7C1_CX4-gaF6FJHrjtykscaHMg4bjwxQKQr2M7Pe86997MnaDcjhAmiOp8-vkHO44A3b80v5pBMwja3uX_SqP2lxS167Yc7VK8IlUwlmlPiJ7sqTdiMsQSu52TjHInOFJCOf0zfIVROTwEtHRyjjE3YVUORl4cp4y6KaLGqlRs1TYSyYdpm73JKETXeHZkG9a7FUpvu8sjC2iR6UhTSPJcNew-zPe2UTW9UmC7KV4dFDAEk4fTCHqYU0VGQsUoxTKYB_xDvMjV2WbjIbmyj7wyOzUt1HoWx8dxBhWD35yhPKFdeoO_5aQqr0JZmEl-Rx2zwkXYxOtL-7FsjMgkFe2MDB9fKBec&refId=cOrIy7NZSSTjPN%2B3uv6jIA%3D%3D&trackingId=UGQKbB%2FBZ7GP9JJtstPY0w%3D%3D&trk=flagship3_search_srp_jobs



https://www.linkedin.com/jobs/view/3757704739/?eBP=CwEAAAGLsCWKg7fbtxXqAjyoRuG9MRSSUIoR6wooLDD4QSOqWFXntsZKjA3zim1TXWiDY_J_pLy15ZB6J8VeLbpPolYUWUA9I6AGu7S8I_nWbB3xQ4pnYJEaxGuL1jRux6PAkAhFrq2rxgKRvmFiLhdgqoxHp10rrtwC8MFNQTW1tt88jjnCY49e17T_UTQSG6V9Irb4rWtgcNnCwifJPmDeiolV80WLOEXsgpOhDWR76YKEcd-xXJu20MRJeMAxVkjwu8Ckt6PhrTvQh5IPBNIgD3M3v6ns9V0p4xwYSB0-8YpPxk9bBkbUjZ4HhamTbcZbA666JB2ccZQVO5O5TMXbTAwHqd83q3pTvDQBJ9sh5cqlb6np6TEaIoGmKjgW_0tWJFBvNW8&refId=vtNyClKWhb0TndvfAlZplw%3D%3D&trackingId=8zlkbqNxY2QjJzXYVNaViw%3D%3D&trk=flagship3_search_srp_jobs



https://www.linkedin.com/jobs/view/3757720867/?eBP=JOB_SEARCH_ORGANIC&refId=cOrIy7NZSSTjPN%2B3uv6jIA%3D%3D&trackingId=FdlP3EXMkSmruxpyK0eyiw%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3758291140/?eBP=JOB_SEARCH_ORGANIC&refId=cOrIy7NZSSTjPN%2B3uv6jIA%3D%3D&trackingId=mKrcXON51Tc53G%2BKIj7NqA%3D%3D

https://www.linkedin.com/jobs/view/3758305381/post-apply/default/?eBP=JOB_SEARCH_ORGANIC&postApplyJobId=3758305381&refId=cOrIy7NZSSTjPN%2B3uv6jIA%3D%3D&trackingId=56hJ1yriJcsYk1LPEqG%2FCA%3D%3D

https://www.linkedin.com/jobs/view/3757403826/?eBP=CwEAAAGLsCI5KZQuaWMfywcoFYE4FNDOG6ZzdGiQEkn8eTFeysE2Zd0ROBDxmMVVIqzQOvKItatEShwIvLaUBOXf6xsJ_eUh1zmm-zdPrMYwmqQ2yo479Qg3ZwKGaKpsX20dZmvn0p2x7jD9kbk2u0rkDn4HCxtjRkBgyaYlVjuptEZxMZJ1fLORzBhgdadq_SU1EEgPOVCLPjngdwP7WMQ6Rq4cEWgcBZoqRDRSKPcYn-X--H2vRhaSSFrF0ARfE9gTysYT9CeVLbNNG6vdI5vj7-eIDXvW7xkP4ojABzZjfTdr7fxMsbDFE0hd0f8EqtvSs_-2IHCd6_bwV8xfEDZ-NGLFCGXdnMdwZwC1ZL8wzAJ0bUIRwFKfTvS7PIcIjApVL4wQyH8&refId=cOrIy7NZSSTjPN%2B3uv6jIA%3D%3D&trackingId=uJfBoNOmVUPZHQY6oVDp4w%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3757376209/?eBP=JOB_SEARCH_ORGANIC&refId=wS3g0VT7HlypmJxWpsMjSg%3D%3D&trackingId=SuN64F%2Bf6qNvZmj8tA8EGg%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3756318650/?eBP=JOB_SEARCH_ORGANIC&refId=VeEUpzF9Bktiu6wP36LgZw%3D%3D&trackingId=KqqJbREQQ0StzjMqAHceaw%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3756322640/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=2V8U1Js9ZMVxrHcgdyxneQ%3D%3D&trackingId=7O8BS7kd0wEpHUe6nV5pAQ%3D%3D

https://www.linkedin.com/jobs/view/3759374185/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=YksXSKNfOAzuARPde6Tkkw%3D%3D&trackingId=KSXMsVeysuPO%2Bah6W5zFFA%3D%3D&trk=flagship3_jobs_discovery_jymbii

https://www.linkedin.com/jobs/view/3752871491/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=YksXSKNfOAzuARPde6Tkkw%3D%3D&trackingId=Uv59jZxBNSWFkqzfhpShVA%3D%3D&trk=flagship3_jobs_discovery_jymbii

https://www.linkedin.com/jobs/view/3756318650/?eBP=JOB_SEARCH_ORGANIC&refId=VeEUpzF9Bktiu6wP36LgZw%3D%3D&trackingId=KqqJbREQQ0StzjMqAHceaw%3D%3D&trk=flagship3_search_srp_jobs

https://www.linkedin.com/jobs/view/3756322640/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=2V8U1Js9ZMVxrHcgdyxneQ%3D%3D&trackingId=7O8BS7kd0wEpHUe6nV5pAQ%3D%3D&trk=flagship3_jobs_discovery_jymbii



https://www.linkedin.com/jobs/view/3755891597/?eBP=CwEAAAGLlvRTc4IgEDW2kZJSYvmx5xRTbfXkG1k25nSuGi6X6LAAXmPKGurcf0HOXjqu6FH2InBO5dfhuUa8EGp34tWaOkL04b2RUZqW5d_kNlYNa46DEdLcnfbk8YVpv2uMUdgLizoIp35fbKCBgkbOCU4dE4SH9e7elvTH8XcfDp0o9fWmfUyf09EqOHWAqv_0BT4VKTGZNjV_oQwHeiF5m_x8ZfzQy6t4_0OV1WxiPLukVJQuCdkVDhdYovkvZwjuYSEr2fJjmVgxqNdx77xQ3XAF_RFRrpPvLtUhWG6_U1Eq13rSLvHqNwyrABMPPMjAbsSvJLZPsKoQVnAjO1qqVumSozsLH0FFzBEAEMca&refId=cfp52M3CI0ifaEhu2PiabA%3D%3D&trackingId=trcj2NN3A3iHHBOQOi98wA%3D%3D&trk=flagship3_jobs_discovery_jymbii


https://www.linkedin.com/jobs/view/3755857524/?eBP=CwEAAAGLsCI-F8xv3yb7t3roTcri9pkMnUomlQ4c_aWIbzNapo86xl32k4L1aTDslj1bVa91fCZ-j5uSne9cRBkkY1yUIotygN8ikJyU8E_VZtkirRFVj8zotigJNdjdDG6SWQPCjHyohB_FLJurHGlCocw7bgANSxmz4n0ZPDXp4FxJR7G37BmEXQYOZkIyF_uN5ZS5vLo1nbfgyBEuz4BmslN-_36vA5xneQpRu_WWs3PpCmWKcx6xJVf5WhMBXZRl6W9CUWhPQBKrkwZlEb_7mt3VCdXMAJyYnNtz3IeSkhVeC7ifbCfVrjesJu1yg0Rn11yDyvwFhw0uxdkVYT3pE_SHDxbj_I59DfJakY5sIirnmd0aRKmhE0HDCYc5YLSuiGhS&refId=bieE3t6uhTKNPQkZIingdg%3D%3D&trackingId=5HGtr2e%2F3UTOk3o9YpKBDw%3D%3D&trk=flagship3_search_srp_jobs


https://www.linkedin.com/jobs/view/3755398857/?eBP=JOB_SEARCH_ORGANIC&refId=Z184NtwtEM0RQJ0GAA%2BKQQ%3D%3D&trackingId=oliIGE51Ap0JJT5kHYlXwg%3D%3D



https://www.linkedin.com/jobs/view/3755420448/?eBP=JOB_SEARCH_ORGANIC&refId=P6k%2FSgBPIKPdbFVIufjDSg%3D%3D&trackingId=nNo3Ht27BO3qj95cg175Iw%3D%3D&trk=flagship3_search_srp_jobs


https://www.linkedin.com/jobs/view/3754991098/?eBP=CwEAAAGLsCK6FXRtLMbkcct7D4ZrdKQJUcwGI7OobppxU_ZoUXGO0jQaYUI4UgP5VLSM8AYKqeuAHB-fdsJCPDqmiymEWctuODiW0o80Ij0aAa2Ih3hB95qA20MLM9xV5anIYAePu8Ie7Qtz0REkNY1fsrKB61ZpRQeSPj9TjQQygLmcncdr7EWoUypdN_DTbLyTtoAte9DMq63wYZI2uR5zEC4NBXRkdvHKz4eBgKfOzVMuOJaMWiL8wDt-LDdn_LSfgLIw9eK0U-TeQyTCnWA0C06v6qsuoEFlJTfbV4hfnUaflT4TmufvmWZ2TMU6YwaTmydL3Wvy-CnDArLVGEbfS-E5BIx7rNVxEYaRZSSftFnLXVzLx59AHMiTV9L9zywwVc5C2Pk&refId=P6k%2FSgBPIKPdbFVIufjDSg%3D%3D&trackingId=yfKOGycdJZCmt60pRn6CkA%3D%3D

https://www.linkedin.com/jobs/view/3755135879/?eBP=CwEAAAGLsCTlqDYFifDjkKG_Vk3tHETRgqM_lQLK6bdIcv-NCNfPP270BHg4GHmAYUYVY8jtkPQGFhjKXNpbzQbN8QxAGW0fScEKFBgQ_4vPL8Fr2dNxrGofK3iAqK6j6HY2c6hLU9HivHnqwtzAe9F0SeFTBd7TRZgZllM9aaQbOhBtdB1-jRAk_k8NWYFS9Rf6XANI31xuSjKtCAQt_afQwTwAHgFZgbTZNy7nBMpfXFWVW8r9Z0BD1nC_rsWsCqXt4qITbqOiPClDgcXhwciYG9aHy_z9jK3RAAV3tDX1EZ-fpboJbe4A5jTsMAD34-GBYU5PwmQbs7C7z0lNMxKSW0d_CRewQJMOmRpjiwgpLgcFH1v1-L_1BI420ZQXBRXIr5_cEt8&refId=N35xpZQ31gqhPe2uUn8G6Q%3D%3D&trackingId=B%2F4TmJJeQqSOFMALw7J00A%3D%3D

https://www.linkedin.com/jobs/view/3755197566/?eBP=CwEAAAGLsDhwDagNSsF6GakbLDlNC1dYePWhgfowCtXXO3sbH-XPIx9pssNGTLranPosvEcKU279H6B1SnIM6RA2AhAKB_4g_NuSJfPeoYecp7leZG-NkWwyKOIZMPSwAOtfkW1kyWFBNZvAmTNGzMRVNEf-Apx7Tx4vh_-Smgvfh_JdJAAg38jROyThSeC5k8IZcfWRo8DcrNfembjsAnYOHlUH__tEHwjxfhySVr_8lTmQ_bB-YjhYEeO6MXZQYckMwkz5pK5sa7_0Qc-Szx1ftN2LAeGAHXhuSRTQ3AhqZZy5dGN1PRYVduX7ufLd6V5pI8GG2Uzhk2ifDaLuwkiODCv2vs9hpMCQJluU3hmX-ZqLff1knHDweHlIdkDKpcT1MzQl-Ypiow&refId=5HkQMd%2Byg42KXa2mYtgc9A%3D%3D&trackingId=irK9UMKa5Bs08wQZbhK7sg%3D%3D












"""

url2scrape = split_and_clean(x)
print(len(url2scrape))

29


In [23]:

# run it and save it 
driver = open_browser(info_dict)

bd = '/Users/phil/Library/CloudStorage/Dropbox/GITHUB/scrapifurs/scrapifurs/data/APPLIED_JOBS'
file_name = bd+os.sep+'master_jobs_applied_to.csv'

save_jobs_data(driver, url2scrape, file_name)

print("DONE")
driver.close()



JOB ID: 3752873243
Repeat entry, skipping...
JOB ID: 3761064412
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":".jobs-apply-button.artdeco-button"}
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001031fe004 chromedriver + 4169732
1   chromedriver                        0x00000001031f5ff8 chromedriver + 4136952
2   chromedriver                        0x0000000102e4b500 chromedriver + 292096
3   chromedriver                        0x0000000102e907a0 chromedriver + 575392
4   chromedriver                        0x0000000102ecb818 chromedriver + 817176
5   chromedriver                        0x0000000102e845e8 chromedriver + 525800
6   chromedriver                        0x0000000102e854b8 chromedriver + 529592
7   chromedriver                     

In [ ]:
driver.close()

In [ ]:
asdfasdfasdf

# end 


In [ ]:
"""







https://www.linkedin.com/jobs/view/3748333969/?eBP=CwEAAAGLlvJJcHmEYL9wYPdjGPNLUGiKeWWiA4nFQiFVWhGPg3wFNHgVjA3qcRMDLzK045ZqoiyXK8fz8WmNQuH2Nh-NAkKF29pHxpcR0Lxew6ODavP1FomLpxPT2bXWFcdT77-KqLg-cX2sq_VKO1AVesKR7Cot0UMoKLqrURRDdOqBe15pCscfwS5ENn1o36nTSVvlfwZjY_GHmo-giYPRQ65ilUvjkrsj99UuhJvt_E-cXgatOyYe_X2u_ixWeTmLzHp3egaowbScodLc6w7nLw1CkzjcZFsvvU5LWleCrzq0wF5Ht7dXtmDP7nKi96H0XVGj7WVT8uIpcgfnXOjrXXlwCnJCyB2KvBJ_xtFZ&refId=u1RXW9vjT8Nkdh1GotG7fw%3D%3D&trackingId=0rs9DyJckh5%2B8a5%2Bbl5oCg%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3748194754/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=0BZaxXeMIPTGGyLyf%2BG4%2BQ%3D%3D&trackingId=ZtZl4zIYznDO4pZvKrseuw%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3748125140/?eBP=CwEAAAGLlvDuvbvkOI44PxMMTfE3VasNvyq0UWnhmwqZff1t7Jh342QRFau-_1HikD7rmfxB1PQ2eCri_fPWiQ6eWnMb5iSO1nliyJsEt-OXBhLt5WTGTUgTktoIAbKsyx2uw2nZiT5mvg1mx098-0pb2tZGpmq-VAWm_Af-e9DOx8Cf88ugPrJXUXEerAQCp_PsH8XcgPM0yzRsC4QeNA1ladRdbkDRv_4DEW8adKmfpTQmzFot8gvM-n0Gh5coJYfIz-oNoC2yG8ZV6aXCOv-IojyMrR_xssq3f3cWKzGBarx0K1vstxt5ulNUlLBQQpY7-kWLCAdipMzKpCOG6z30n06Uic5jL2EiZkEiR3t2ALKhBA&refId=kPVSvDcWMC9sWOF%2B%2FjxsnA%3D%3D&trackingId=Psww2ntGL9iALv%2BAbbwJ0g%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3747922772/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=COXW8QCZo7o7xa3Dy7GlEQ%3D%3D&trackingId=PRR74PAlySODuopzNvEM6g%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3747901100/?eBP=CwEAAAGLlvGV3rlkGh2rZl8F1N-WyRGdqYRqcIwsonSiw9m2cizzMSxgX9FLpCsfpri1lhwc7bEAmwNL_yTcyOYXggZT93jp3NZTsQ2ABB1Fz_V068dh1-jUUWzxCabulBdzFmmhyQU9qoM05-b-5Tx1D38-BYq1wJzCqAgnvBb8uiwo4wgG7CgHh2xwxF4TDWH0D3sS6mun2CwCe9VznwdBiec6tZPcep8K9VOqdiZIEElTOlPgIihjJSAkq_WnFRaCjXnzCYlD-0u1iTzttCS6GiemUP1BxR_jVkxS0lYbZA0WijTVe2Ef3ncdCCaiel_Urizqj3-ilr-yW61gdIUwEUhiTvrYcsDG6dMy_onh&refId=NHTDYu5Bb5MRRkrr36FvOw%3D%3D&trackingId=rpx8d1IbOjH%2FCZN%2FdQc9hQ%3D%3D&trk=flagship3_jobs_discovery_jymbii



https://www.linkedin.com/jobs/view/3747684800/?eBP=CwEAAAGLlvDyfVjySaOk5M7MmqVlspk-dlhji9k6xx4Uq_nLHLmTQ7FG1giduki2lOXoY-5EEhK81lANHvK70s9CuXerVjvWEEvNsS0e0RjsnLGcYxQnbFb5DIox3VpjkhvNBGer2pGEBUBX6xx5I_REKEi-xJHwCRDNXhmK1Tr6AukaU0Hbem9GZ3NCtnooc6pvsR83AOnCphItBrfFX6cUUo4KNAn_rGasGODJ5QApQnVbjwpoWdluhnSisvWHR34FH_qzod-9NUFM3m0WHiU2ZklmJNPloqZRqx5hchJs1xsbYU3LjQddouriTqGA39nql9H4xsXfSb9zwCU_mLZDNJSCS4cxP88YZfw43bRnzdHc_A&refId=0BZaxXeMIPTGGyLyf%2BG4%2BQ%3D%3D&trackingId=eQRG6anE8fHy1lbOq4mCrg%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3747591506/?eBP=CwEAAAGLlvDyfe8nxJ7VBh2DpEG6bbo7czTm5vgdx-mT_DGEGQai9Fsp07QCshZqdEZlncZF_XQXnW2aoQ4GAMDAOtou0BLjVq0tiZoDBM86p012aVCoPcKjOaClx-mL3ilqk4fCYPvuva_45wp9PS5vGd45ncp76tS2CLHmdY9xxXnMkfuELAiHEd7HWHd5OnC7eMuigsgtgZZctSFcjeLoasjGb34K0bCZ1htmwWTXPwttrWAc1oDmV0-zQmf94hi_2ajlFaHHGkp3bEDqeEyo0hf1dRd1sBrP0PDPp8r1S2oII5qHOXTmcaqkJ13AZt7C0D3ErvTPQJgIhXUo16lF4XNUmmPEuV3kQw2m2eOoDl3LYA&refId=0BZaxXeMIPTGGyLyf%2BG4%2BQ%3D%3D&trackingId=l7B%2BqBHxhFlUtzMM1BFU0w%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3747575218/?eBP=CwEAAAGLlvRTc6kJKRFAgWGWcnxFvTRAuui7ZfKf46N5ahhoYx6loTaVdXPizeKmDibOTuAslugGciUTZOePdMAi8y_0ckO0jVBZ834A4D17UFuMt5hwGajkQV5ImWjwuFAH31OeKlBhyKP_LfgOErUT3b2H2wA5N2GxwReCtA8_b-wzgQClx9LeEKfVKS2UZMAhBT7C7tQzmz9CrPmUV2-lr4rG2GnucFfHS2kWX-RhXkJo-j8nLldxCBXq7NdhfwImiHKReB0JJ9wcdfEs9JKIz8dYyJRVdNHm3x9y1acxb5BQBEbPhZd96ig0Hrc4jNWEGmTEJthaKGDvQPCVS5085j-Hit1Uj_DvzC_rHG4T&refId=cfp52M3CI0ifaEhu2PiabA%3D%3D&trackingId=ZecK7LetiJ9rScEOn8ojhQ%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3747428977/?eBP=CwEAAAGLlvOkripxUjpccMQQ0gnNKsOEflnWiUhRi21Jmv8658feHBzzZGBRXrbPxCQRv_1LrPGlGKwAK_Vx2CtK159ImpCgkQn66z61w746-2RHWfC2VcVwrfD6k3wjnlgVS9_5mW8W2SYyzgjQdDWKGR-Rh7rYwyFGLraOwpodhazUVqd-T8N0dSEHpXfpMi-bTS2fY4VUZpBhvfXobty-2O06wBq2vJfLcnkA6BwnlfBbSx2f1HEv9CkqT23rqv-65tVQE65GPpVz64vPD724qgi-8DJxfbHZFVuo1NbVCI8BW1oN0NJrAIqBx6FeJGN68moomEHrC8vUDuclQxEY-FrT1BpaT2wwALb_PRM1Xy61yA&refId=LYEk3O9y1%2FOv4gXzWa1QPw%3D%3D&trackingId=x0SV0nPAHktaWnDnfURrGQ%3D%3D&trk=flagship3_jobs_discovery_jymbii



https://www.linkedin.com/jobs/view/3747308265/?eBP=CwEAAAGLlvDyfWVTpvOYmCAbVQNwOGvAawISu6PjGMa5RmpjtbGWS3qdf8VBtpe9BLeZNe6Hq_eNuHDFLZmd9tEftVNMOIGe5uWkOmWjEAUgY0bT_gE9HfVISmL6SLR9ohrZtOnYCvXKW_9lBEhHzMzClrN79id_NWRFZf21Yty6tYYvCzmxCYJaJdPS8e9Egd_LeRh3wKt-8nMx1sLs87cAETjuR9TqVkT4ZIVwSM76zeWxNDu6ZcrkmOEUbZG7xGxPgc3Z8eBTHf-WZ3rloQFTTLO-EhBqyj-ZdW4YhNsO2XplMopzF7dD9pmbDVQLq-JYcMKtX2LS7MTKObyrMRRVuitusUzsLpF-ieWvgd71sTEfog&refId=0BZaxXeMIPTGGyLyf%2BG4%2BQ%3D%3D&trackingId=nG559qdKttC6sys8bJtdFA%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3746850937/?eBP=CwEAAAGLlvRTcyDCsgqn62hzsbq53C7GcuozllKtXnyzu3EUdNLD76uENUQIEdQiKvU49Dh6ReZHPm5ZkcF6nP8wkBON4bo3MU3KTeO52xXwPqxnXf3AGjhkUFnIXkQOqjMdXZ_RyQExdPAMzIObUE95krGJOYah3_GQzel8h_oyi1EWzQT5M-n9_Kks2Em5bNr5d6QctC7U5x4Jt6W23HYvF5Tqy4N7IUz7i2AfvwKv1tMuRwIIo5QUJdz5aPP9nO28ku0sdNpe1I5Jqh0tY1gUR_kPzhOWBtaUuX4Hyq8wQ1h9eecd9bnFkF3I5_XbmrcYzDVp2zuQgwT-vnfZ3QbpVRybbFmZ-EPE4sfQUfS3sq4&refId=cfp52M3CI0ifaEhu2PiabA%3D%3D&trackingId=mahwscnuI0gzO%2BXPGZvG8Q%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3746777426/?eBP=CwEAAAGLlvUGlvoCK4L62-rBQ6CVWaUwnQk4mdN6EWEmF9I_VWKMx47G8QcpD2i9_741uLrjULlDm9IVeqmfNMKonqIdJd-drdHqinse4WnqXVW-Ju8cM9ZydbRWNT7Yh7wMuOC23jKzCw1bX8_4juXHpKOZsh6nxDk8eUDqy-uZ_ng7q444fuKBV4MoWVrdLKj7lWY7BEDL3ObS0GSfWVDxZr1GDUV-Qv38T5nZcVvxicYtY3mCTTyfuwwkvme5vvjtYpt3Qc5n0UlF1TAO-LvfO1gIHedBdMJZ_uWHk48K3N1bZNeTahrfamsg5l7uevzP3SgpffwZR9RzpN1t-NG6OcncK7okqRqgM0yoBl9bFY_M7g&refId=fVnYs9V%2BAJf7tNQwxRl5lw%3D%3D&trackingId=8Dob7XMS5XBumOfkMGnPIw%3D%3D&trk=flagship3_jobs_discovery_jymbii
https://www.linkedin.com/jobs/view/3746605654/?eBP=CwEAAAGLlvJJcJqj3hWjyN7cvIpPa8Q6hcm0gnWE0OMVSvm-NuPHoyIm3a3oH_UGd_IzYowz-akfbm70aakBguGZ7Oa5cubQk4tLWS-ze5o5t_1X_o3nbK8s2efy1Qv_mWVO65hnhdYU_J6_g9-mknLiJwQUORSSzzRdgmYzp0nOBBEtUbMQ0CWtXxuAmLKUESahu4Xs5PbwVpAB3XfnQz3p721CdahfHs03znV724FowUtRD05maFPopD4qjrRxuBPtYnXslk1hjqJ6mAQ-3lSz52k46Z4k8d-09WSHaSRjn9EbtVk-TK8nk5T9WVMkhXBLCmrf6OiVHfJX2B_QAGHrznWN2ZJmS5kmjdbVJr-6&refId=u1RXW9vjT8Nkdh1GotG7fw%3D%3D&trackingId=xHRkyKvq5B4UVdmo6rnFfQ%3D%3D&trk=flagship3_jobs_discovery_jymbii

"""

In [249]:
x = """


"""

url2scrape = split_and_clean(x)

In [251]:
# quick_check_if_applied

for linked_in_link in url2scrape:
    
    # Extracting data from the provided HTML
    job_id_match = re.search(r'/view/(\d+)/', linked_in_link)
    job_id = int(job_id_match.group(1)) if job_id_match else -1
    # check if it in there first to avoid repeating all the process
    is_repeat = check_job_id(file_name, job_id)
    print(f'JOB ID: {job_id}')
    if not is_repeat:
        print(linked_in_link)

JOB ID: 3750186401
https://www.linkedin.com/jobs/view/3750186401/?eBP=CwEAAAGLlvDuvfWytbFwIntdq4ZIIHRY82g9S_7pXRjW2kjWMYGCKW4XgnWcIVRS5LKaIdX-OzjlNTWKKoizAH4qe7_mqcKVq2MQCEG5qMaqN_nvBfXwtdq7DQmXAKSzVy8u9Whm5DaK3I6uq9K17ojao78pJaMCJcSnlHutjbWdQ4Am-M0Uu62YeOG4Ybj-b4GiZGjybGGVbMv8f8lzp7jhukZR8CERFKt-PNpr6hDJb2IOmxL98kgENPff4PPZw2ibdzPEUN7uHe9wsztWHTVKAOqFI7ZQQvEptJSDeqclLf2KZdrSWoeYA67v7Eyce7mU064&refId=kPVSvDcWMC9sWOF%2B%2FjxsnA%3D%3D&trackingId=65tTtyewctT2LNCYsYyGjQ%3D%3D&trk=flagship3_jobs_discovery_jymbii
False
JOB ID: 3750181212
https://www.linkedin.com/jobs/view/3750181212/?eBP=CwEAAAGLlvRTcyjk7rxvuZ3t9u71jkyyWx9NJOE1t_ZxGq5PzHMjXIuWQFh3FxWg0RqZM6zTmx29WxNOdCvRAeR29skMBnBfuQfylNXUgLoXImf4gneemCIP5QZgVqw0iXj6SACSnQPcUFta1oBeKP2xBudaBxYdhv8C866NUy5ezQqs3Ey27Cg-P7KmhIp_h_kqFkN5u3dYQRsvGelRuAQwiWssJN_vOvidTjtKp-zTx5J6CdBS9VQ_IyxxpUnFU2KQLu6TQAmms_PjrEHhLv7ybBlMA7bId9afUjAu9HnP2LEsmVIp6ZtfKq-uD2Z8ymJwG_GO4XjRIDCK6g__Cg8hHLI1zJE5kHA095xJZqDs&refId=cfp52M3CI0ifaEhu2PiabA%3D%3D&trackingId=

In [ ]:
asdfasdfasdfasdf

In [171]:

    


bd = '/Users/phil/Library/CloudStorage/Dropbox/GITHUB/scrapifurs/scrapifurs/data/APPLIED_JOBS'
file_name = bd+os.sep+'master_jobs_applied_to.csv'

update_csv(df, file_name)


In [167]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# def click_show_more_button(driver):
#     # Wait for the button to be clickable
#     wait = WebDriverWait(driver, 2)
#     see_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ember33"]')))
    
#     # Scroll the button into view
#     driver.execute_script("arguments[0].scrollIntoView(true);", see_more_button)
    
#     # Add a slight delay
#     time.sleep(1)

#     # Try clicking with Selenium first
#     try:
#         see_more_button.click()
#     except:
#         # If that fails, use JavaScript to click the button
#         driver.execute_script("arguments[0].click();", see_more_button)

# click_show_more_button(driver)


In [168]:


click_show_more_button(driver)



In [156]:
wait = WebDriverWait(driver, 2)
see_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ember33"]')))
see_more_button.click()



ElementClickInterceptedException: Message: element click intercepted: Element <button aria-label="Click to see more description" aria-expanded="false" id="ember33" class="jobs-description__footer-button t-14 t-black--light t-bold artdeco-card__action artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--fluid artdeco-button--tertiary ember-view" data-artdeco-is-focused="true">...</button> is not clickable at point (435, 20). Other element would receive the click: <div id="global-nav-search" class="global-nav__search
            ">...</div>
  (Session info: chrome=118.0.5993.117)
Stacktrace:
0   chromedriver                        0x0000000101148510 chromedriver + 4310288
1   chromedriver                        0x00000001011404bc chromedriver + 4277436
2   chromedriver                        0x0000000100d73b6c chromedriver + 293740
3   chromedriver                        0x0000000100dbfcb4 chromedriver + 605364
4   chromedriver                        0x0000000100db

In [150]:
wait = WebDriverWait(driver, 1)

show_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show more')]")))
show_more_button.click()


# see_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "Click to see more description")]')))
# see_more_button.click()


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000101148510 chromedriver + 4310288
1   chromedriver                        0x00000001011404bc chromedriver + 4277436
2   chromedriver                        0x0000000100d73b6c chromedriver + 293740
3   chromedriver                        0x0000000100db9040 chromedriver + 577600
4   chromedriver                        0x0000000100df3e60 chromedriver + 818784
5   chromedriver                        0x0000000100dacfd0 chromedriver + 528336
6   chromedriver                        0x0000000100dade7c chromedriver + 532092
7   chromedriver                        0x000000010110e834 chromedriver + 4073524
8   chromedriver                        0x00000001011127fc chromedriver + 4089852
9   chromedriver                        0x0000000101112c58 chromedriver + 4090968
10  chromedriver                        0x00000001011188f8 chromedriver + 4114680
11  chromedriver                        0x0000000101113234 chro

In [98]:
for key in df:
    print(key + '  ' + str(type(df[key][0])))

linked_in_link  <class 'str'>
job_id  <class 'numpy.int64'>
job_title  <class 'str'>
company_name  <class 'str'>
apply_link  <class 'str'>
job_location  <class 'str'>
time_since_posted  <class 'str'>
num_applicants  <class 'str'>
salary_range  <class 'str'>
job_type  <class 'str'>
job_level  <class 'str'>
company_info  <class 'str'>
description  <class 'str'>
matched_skills  <class 'list'>
unmatched_skills  <class 'list'>
date_number  <class 'numpy.int64'>
formatted_date  <class 'str'>


AttributeError: 'DataFrame' object has no attribute 'itercols'

In [84]:
# import re

# def get_job_id(element, css_selector):
#     content = safe_extract(element, css_selector)
    
#     # Use regex to find the job ID number from the content
#     job_id_match = re.search(r'/voyager/api/jobs/jobPostings/(\d+)', content)
#     if job_id_match:
#         return job_id_match.group(1)
#     return None

# # Use the function

# job_id = get_job_id(driver, '#datalet-bpr-guid-3188948')


In [49]:
def get_element_data(element):
    data = {
        "tag": element.tag_name,
        "class": element.get_attribute('class'),
        "id": element.get_attribute('id'),
        "text": element.text.strip(),
        "children": []
    }

    for child in element.find_elements_by_xpath("./*"):
        if "job" in child.get_attribute('class').lower() or \
           "job" in child.get_attribute('id').lower() or \
           "job" in child.get_attribute('name').lower():
            child_data = get_element_data(child)
            data["children"].append(child_data)
            # Removing child's text from parent to avoid duplication
            data["text"] = data["text"].replace(child_data["text"], '').strip()

    return data

elements_with_job = WebDriverWait(driver, 1).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        "[class*='job'], [id*='job'], [name*='job']"))
)

all_elements_data = []

for element in elements_with_job:
    all_elements_data.append(get_element_data(element))

print(all_elements_data)


AttributeError: 'WebElement' object has no attribute 'find_elements_by_xpath'

In [ ]:

def get_job_details(driver):
    job_dict = {
        "job_ids": [],
        "is_promoted": [],
        "job_title": [],
        "company_name": [],
        "Location": [],
        "pay": [],
        "job_link": [],
        "time_added": [],
    }
    
    jobs = driver.find_elements(By.CSS_SELECTOR, '[data-occludable-job-id]')
    
    for job in jobs:
        try:
            job_id = job.get_attribute("data-occludable-job-id")
        except NoSuchElementException:
            job_id = -1
        
        try:
            promoted_elements = job.find_elements(By.CSS_SELECTOR, '.job-card-container__footer-item')
            is_promoted = "Promoted" in [e.text for e in promoted_elements]
        except NoSuchElementException:
            is_promoted = False

        try:
            job_title = job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text
        except NoSuchElementException:
            job_title = ""

        try:
            company_name = job.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description').text
        except NoSuchElementException:
            company_name = ""

        try:
            Location = job.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            Location = ""

        try:
            pay = job.find_element(By.CSS_SELECTOR, '.mt1 .job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            pay = ""
            
        try:
            job_link = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link").get_attribute("href")
        except NoSuchElementException:
            job_link = ""
            
        

        if job_title != "" and company_name != "":
            job_dict["job_ids"].append(job_id)
            job_dict["is_promoted"].append(is_promoted)
            job_dict["job_title"].append(job_title)
            job_dict["company_name"].append(company_name)
            job_dict["Location"].append(Location)
            job_dict["pay"].append(pay)
            job_dict["job_link"].append(job_link)
            
            current_time = datetime.now()
            formatted_time = current_time.strftime('%Y%m%d%H%M%S')
            formatted_time_as_int = int(formatted_time)
            job_dict["time_added"].append(formatted_time_as_int)
            
        
    return job_dict

#### navigate to the first page we want to scrape

In [28]:

# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3481251151&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C288%2C2463%2C25584&f_WT=1&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3682851114&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currefntJobId=3642657953&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r86400&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3737611604&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r604800&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# # x = 'https://www.linkedin.com/jobs/search/?currentJobId=3729935051&distance=25&f_PP=102277331%2C102448103%2C106233382%2C103918656%2C100075706%2C102250832%2C103575230&f_TPR=r86400&f_WT=1%2C3%2C2&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3741083392&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C25584%2C2463%2C288%2C30128&f_TPR=r86400&f_WT=1%2C2%2C3&geoId=103644278&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD'
# info_dict['start_url'] = x
# driver.get(info_dict['start_url'])
# time.sleep(5)


In [29]:
# df_main = pd.read_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv')
# # df_main['job_ids'] = df_main['job_ids'].astype(int)
# # df_main.to_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv', index=False)



In [27]:
# # job_data = get_job_details(driver)
# # df_main = pd.DataFrame(job_data)
# # df_main

# n_pages_to_scrape = 5
# wait_sec_each_page = 10
# update_every_n_secs = 60*6





# while True:
#     for n_page in range(n_pages_to_scrape):
#         if n_page ==0:
#             # go back to the first page
#             driver.get(info_dict['start_url'])
#         else:
#             go_to_next_page(driver)
#         time.sleep(wait_sec_each_page)
#         # update data
#         job_data = get_job_details(driver)
#         job_data = pd.DataFrame(job_data)
#         job_data['job_ids'] = job_data['job_ids'].astype('int')
#         df_main = update_dataframe(df_main, job_data, ['job_ids'])
    
#     # save CSV 
#     df_main.to_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv', index=False)
    
#     print('_____________________________________________\n\n____________________')
#     time.sleep(update_every_n_secs)


In [172]:
x = """
About the job
At Expa we‚Äôre dedicated to being the best place for entrepreneurs to create and build companies. Expa was founded in 2013 by Uber co-founder Garrett Camp to help bring great ideas to life. We‚Äôre builders, founders, and operators ourselves, who know the struggles that come with starting a new company, and what it takes to succeed. We support entrepreneurs with more than just funding ‚Äî we offer a valuable network, office space, and our advice and expertise. Expa Studio combines technical and design expertise, operational support, and capital to turn bold ideas into category-defining companies.

Your role

We are looking for a Data Scientist at a new studio company tasked with research and financials around enterprise businesses. They will support our product, sales, and leadership teams with insights gained and tying that back to a specific product idea (e.g. understanding business fundamentals of different market verticals across a matrix of investment and operational criteria). The key to success in this role is conducting superior analysis, modeling data, translating that analysis into informed decisions, and then testing those decisions in a live environment.

The ideal candidate is adept at using large data sets to find opportunities for product and process optimization and using models to test the effectiveness of different courses of action. They must have strong experience using a variety of data mining and data analysis methods. The candidate is well-versed in using a variety of data tools, building and implementing models, using/creating algorithms and creating/running simulations. They must have a proven ability to drive business results with their data-based insights. They must be comfortable working with a wide range of stakeholders and functional teams. The right candidate will have a passion for discovering solutions hidden in large data sets and working with stakeholders to improve business outcomes.

Your recommendations, based on sound analytics and quantitative modeling will help determine how we improve our products. You‚Äôll be working with a world-class team who have built and shipped products valued over $8B today.

Responsibilities

Work with stakeholders throughout the organization to identify opportunities and leverage that data to drive business solutions.
Mine and analyze data from databases to drive optimization and improvement of product development, marketing techniques and business strategies.
Assess the effectiveness and accuracy of new data sources and data gathering techniques.
Develop custom data models and algorithms to apply to data sets.
Use predictive modeling to increase and optimize customer experiences, revenue generation, ad targeting and other business outcomes.
Coordinate with different functional teams to implement models and monitor outcomes.
Develop processes and tools to monitor and analyze model performance and data accuracy.
Conduct research, compile about industry and customer trends
Help identify workflows and processes
Apply your expertise in analysis, data mining, and the presentation of data to see beyond the numbers and understand how our potential users will interact with our products
Provide analytical support to stakeholders across various fields (Product, Eng, Design, UX, etc.).
Create financial and business models, matrices and other basic tools to help team members analyze business needs.


Qualifications

Strong problem solving skills with an emphasis on product development.
Experience using statistical computer languages (R, Python, SQL, etc.) to manipulate data and draw insights from large data sets.
Experience working with and creating data architectures.
Knowledge of a variety of machine learning techniques (clustering, decision tree learning, artificial neural networks, etc.) and their real-world advantages/drawbacks.
Knowledge of advanced statistical techniques and concepts (regression, properties of distributions, statistical tests and proper usage, etc.) and experience with applications.
Excellent written and verbal communication skills for coordinating across teams.
A drive to learn and master new technologies and techniques.


Your Experience

You are a driven professional with a degree in Statistics, Mathematics, Computer Science or another quantitative field 
2-3 years of experience manipulating data sets and building statistical models, 
Coding knowledge and experience with several languages including: C, C++, Java, JavaScript, etc.
Knowledge and experience in statistical and data mining techniques: GLM/Regression, Random Forest, Boosting, Trees, text mining, social network analysis, etc.
Experience querying databases and using statistical computer languages: R, Python, SLQ, etc.
Experience creating and using advanced machine learning algorithms and statistics: regression, simulation, scenario analysis, modeling, clustering, decision trees, neural networks, etc.
Experience analyzing data from 3rd party providers: Google Analytics, Site Catalyst, Coremetrics, Adwords, Crimson Hexagon, Facebook Insights, etc.
Experience with distributed data/computing tools: Map/Reduce, Hadoop, Hive, Spark, Gurobi, MySQL, etc.
Experience visualizing/presenting data for stakeholders using: Periscope, Business Objects, D3, ggplot, etc.
Excellent written and verbal communication skills
Attention to detail


This will be a full-time position with a salary range of $100,000 - $120,000, which is determined by various factors, including, but not limited to, work experience, skills, location, depth and maturity of our project. We're a small team and we want to try and spend as much time iterating during these early days of the idea. Because of this, the expectation for the Data Scientist position is that you will come into the West Hollywood office daily (M-F).

"""

print(x)


About the job
At Expa we‚Äôre dedicated to being the best place for entrepreneurs to create and build companies. Expa was founded in 2013 by Uber co-founder Garrett Camp to help bring great ideas to life. We‚Äôre builders, founders, and operators ourselves, who know the struggles that come with starting a new company, and what it takes to succeed. We support entrepreneurs with more than just funding ‚Äî we offer a valuable network, office space, and our advice and expertise. Expa Studio combines technical and design expertise, operational support, and capital to turn bold ideas into category-defining companies.

Your role

We are looking for a Data Scientist at a new studio company tasked with research and financials around enterprise businesses. They will support our product, sales, and leadership teams with insights gained and tying that back to a specific product idea (e.g. understanding business fundamentals of different market verticals across a matrix of investment and operation

In [ ]:
# test scraping full job page for saving data for all jobs I apllied for 

In [145]:
# x = 'https://www.linkedin.com/jobs/view/3743737484/?eBP=JOB_SEARCH_ORGANIC&refId=wIC2skgivSr27kABYstWjg%3D%3D&trackingId=XVrPkXaydKf4xGrJyLPUOw%3D%3D'
# info_dict['start_url'] = x
# driver.get(info_dict['start_url'])



In [25]:
# from selenium.common.exceptions import NoSuchElementException

# def get_job_detail(driver):
#     job_detail_dict = {
#         "job_title": "",
#         "company_name": "",
#         # ... add other fields as necessary
#     }

#     # Scrape the job title
#     try:
#         job_detail_dict["job_title"] = driver.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.job-details-jobs-unified-top-card__job-title').text.strip()
#     except NoSuchElementException:
#         job_detail_dict["job_title"] = ""

#     # Scrape the company name
#     try:
#         job_detail_dict["company_name"] = driver.find_element(By.CSS_SELECTOR, 'a.app-aware-link').text.strip()
#     except NoSuchElementException:
#         job_detail_dict["company_name"] = ""

#     # ... similarly add extraction logic for any other information you need

#     return job_detail_dict

# get_job_detail(driver)

{'job_title': 'Data Scientist', 'company_name': ''}

In [60]:
# # Function to safely fetch elements
# def safe_extract(selector):
#     try:
#         return driver.find_element(By.CSS_SELECTOR, selector).text
#     except Exception as e:
#         return f"Error: {e}"

# # Extract contents with updated selectors
# job_title = safe_extract('h1')  # This one works
# company_info = safe_extract('.job-details-jobs-unified-top-card__subtitle span:nth-child(1)')  # Adjusting selector
# job_description = safe_extract('.description')  # Using a more common class name for job descriptions

# print(job_title)
# # print(company_info)
# # print(job_description)


Data Scientist


In [44]:

# # Define a function to safely extract elements
# def safe_extract(element, css_selector, attribute=None):
#     try:
#         if attribute:
#             return element.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
#         return element.find_element(By.CSS_SELECTOR, css_selector).text
#     except NoSuchElementException:
#         return None

# # Extracting data from the provided HTML
# job_title = safe_extract(driver, '.job-details-jobs-unified-top-card__job-title')
# company_name = safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description a')
# company_logo_url = safe_extract(driver, '.ivm-view-attr__img--centered', 'src')
# job_location = safe_extract(driver, '.job-details-jobs-unified-top-card__primary-description span:nth-child(2)')
# time_since_posted = safe_extract(driver, '.tvm__text--neutral span')
# num_applicants = safe_extract(driver, '.tvm__text--neutral:last-child')
# salary_range = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight span span:first-child')
# job_type = safe_extract(driver, '.ui-label.ui-label--accent-3:nth-child(1) span:first-child')
# job_level = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight-view-model-secondary:nth-child(3)')
# company_info = safe_extract(driver, '.job-details-jobs-unified-top-card__job-insight:nth-child(2) span')

# job_description_element = ".jobs-description-content__text"
# description = safe_extract(driver, job_description_element)




# print(f"Job Title: {job_title}")
# print(f"Company Name: {company_name}")
# print(f"Company Logo URL: {company_logo_url}")
# print(f"Job Location: {job_location}")
# print(f"Time Since Posted: {time_since_posted}")
# print(f"Number of Applicants: {num_applicants}")
# print(f"Salary Range: {salary_range}")
# print(f"Job Type: {job_type}")
# print(f"Job Level: {job_level}")
# print(f"Company Info: {company_info}")
# print(description)

In [45]:
# job_description_element = ".jobs-description-content__text"
# description = safe_extract(driver, job_description_element)

# # print(description)

In [46]:
# from selenium.webdriver.common.by import By

# # Use the find_element method with By.CSS_SELECTOR to get the location
# location_div = driver.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__primary-description div")
# location_text = location_div.text

# # Since the location is preceded by a `·`, we'll split by it and take the second item
# job_location = location_text.split('·')[1].strip()
# print(job_location)  # Outputs: Los Angeles, CA
